## DC-GAN

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline
import tensorflow 
import os
import glob
import imageio
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display

os.environ["CUDA_VISIBLE_DEVICES"]="0"


# In[3]:


images1 = [cv2.resize(cv2.imread(file),(56,56)) for file in glob.glob('IMagenet/tiny-imagenet-200/val/images/*.JPEG')]
images2t=[cv2.resize(cv2.imread(file),(56,56)) for file in glob.glob('IMagenet/tiny-imagenet-200/test/images/*.JPEG')]
images2=images2t[:9900]


# In[4]:


train_images=np.vstack((images1,images2))


# In[5]:


#train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]


# In[6]:





# In[7]:


images=train_images


# In[8]:


plt.imshow(train_images[1],cmap='gray')


# In[9]:


random_input = tensorflow.keras.layers.Input(shape = 100)

x = tensorflow.keras.layers.Dense(128 * 5 * 5)(random_input)
x = tensorflow.keras.layers.Activation('relu')(x)
x = tensorflow.keras.layers.BatchNormalization(momentum=0.8)(x)

x = tensorflow.keras.layers.Reshape((5, 5, 128))(x)

x = tensorflow.keras.layers.Conv2DTranspose(filters=128, kernel_size=(3,3), strides=(2,2))(x)
x = tensorflow.keras.layers.Activation('relu')(x)
x = tensorflow.keras.layers.BatchNormalization(momentum=0.8)(x)

x = tensorflow.keras.layers.Conv2DTranspose(filters=128, kernel_size=(3,3), strides=(2,2))(x)
x = tensorflow.keras.layers.Activation('relu')(x)
x = tensorflow.keras.layers.BatchNormalization(momentum=0.8)(x)

x = tensorflow.keras.layers.Conv2DTranspose(filters=128, kernel_size=(3,3), strides=(2,2))(x)
x = tensorflow.keras.layers.Activation('relu')(x)
x = tensorflow.keras.layers.BatchNormalization(momentum=0.8)(x)

x = tensorflow.keras.layers.Conv2DTranspose(filters=128, kernel_size=(4,4))(x)
x = tensorflow.keras.layers.Activation('relu')(x)
x = tensorflow.keras.layers.BatchNormalization(momentum=0.8)(x)

x = tensorflow.keras.layers.Conv2DTranspose(filters=128, kernel_size=(5,5))(x)
x = tensorflow.keras.layers.Activation('relu')(x)
x = tensorflow.keras.layers.BatchNormalization(momentum=0.8)(x)

x = tensorflow.keras.layers.Conv2DTranspose(filters=128, kernel_size=(5,5))(x)
x = tensorflow.keras.layers.Activation('relu')(x)

x = tensorflow.keras.layers.Conv2D(filters=3, kernel_size=(3,3))(x)
generated_image = tensorflow.keras.layers.Activation('tanh')(x)

generator_network = tensorflow.keras.models.Model(inputs=random_input, outputs=generated_image)
generator_network.summary()


# In[10]:


image_input = tensorflow.keras.layers.Input(shape=(56, 56, 3))

x = tensorflow.keras.layers.Conv2D(filters=128, kernel_size=(3,3))(image_input)
x = tensorflow.keras.layers.LeakyReLU(alpha=0.2)(x)

x = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=2)(x)
x = tensorflow.keras.layers.LeakyReLU(alpha=0.2)(x)
x = tensorflow.keras.layers.BatchNormalization(momentum=0.8)(x)

x = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=2)(x)
x = tensorflow.keras.layers.LeakyReLU(alpha=0.2)(x)
x = tensorflow.keras.layers.BatchNormalization(momentum=0.8)(x)

x = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(5,5))(x)
x = tensorflow.keras.layers.LeakyReLU(alpha=0.2)(x)
x = tensorflow.keras.layers.BatchNormalization(momentum=0.8)(x)

x = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(5,5))(x)
x = tensorflow.keras.layers.LeakyReLU(alpha=0.2)(x)
x = tensorflow.keras.layers.BatchNormalization(momentum=0.8)(x)

x = tensorflow.keras.layers.Flatten()(x)

x = tensorflow.keras.layers.Dense(1)(x)
real_vs_fake_output = tensorflow.keras.layers.Activation('sigmoid')(x)

discriminator_network = tensorflow.keras.models.Model(inputs=image_input, outputs=real_vs_fake_output)
discriminator_network.summary()


# In[11]:


adam_optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)
discriminator_network.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])


# In[12]:


discriminator_network.trainable=False

g_output = generator_network(random_input)
d_output = discriminator_network(g_output)

dcgan_model = tensorflow.keras.models.Model(random_input, d_output)
dcgan_model.summary()


# In[13]:


dcgan_model.compile(loss='binary_crossentropy', optimizer=adam_optimizer)


# In[14]:


indices = [i for i in range(0, len(images))]

def get_random_noise(batch_size, noise_size):
    random_values = np.random.randn(batch_size*noise_size)
    random_noise_batch = np.reshape(random_values, (batch_size, noise_size))
    return random_noise_batch

def get_fake_samples(generator_network, batch_size, noise_size):
    random_noise_batch = get_random_noise(batch_size, noise_size) 
    fake_samples = generator_network.predict_on_batch(random_noise_batch)
    return fake_samples

def get_real_samples(batch_size):
    random_indices = np.random.choice(indices, size=batch_size)
    real_images = images[np.array(random_indices),:]
    return real_images

''''def show_generator_results(generator_network):
    for k in range(9):
        plt.figure(figsize=(11, 11))
        fake_samples = get_fake_samples(generator_network, 9, noise_size)
        fake_samples = (fake_samples+1.0)/2.0
        for j in range(9):
            plt.subplot(990 + 1 + j)
            plt.imshow(fake_samples[j])
            plt.axis('off')
            #plt.title(trainY[i])
        plt.show()
    return'''


# In[ ]:





# In[15]:


epochs = 100
batch_size = 100
steps = 500
noise_size = 100

for i in range(0, epochs):
    #if (i%10 == 0):
        #show_generator_results(generator_network)
    for j in range(steps):
        fake_samples = get_fake_samples(generator_network, batch_size//2, noise_size)
        real_samples = get_real_samples(batch_size=batch_size//2)

        fake_y = np.zeros((batch_size//2, 1))
        real_y = np.ones((batch_size//2, 1))
        
        input_batch = np.vstack((fake_samples, real_samples))
        output_labels = np.vstack((fake_y, real_y))
        
        # Updating Discriminator weights
        discriminator_network.trainable=True
        loss_d = discriminator_network.train_on_batch(input_batch, output_labels)
        
        gan_input = get_random_noise(batch_size, noise_size)
        
        # Make the Discriminator belive that these are real samples and calculate loss to train the generator
        gan_output = np.ones((batch_size))
        
        # Updating Generator weights
        discriminator_network.trainable=False
        loss_g = dcgan_model.train_on_batch(gan_input, gan_output)
        
        if j%50 == 0:
            print ("Epoch:%.0f, Step:%.0f, D-Loss:%.3f, D-Acc:%.3f, G-Loss:%.3f"%(i,j,loss_d[0],loss_d[1]*100,loss_g))
            


# In[ ]:





# In[23]:


''''def get_random_noise(batch_size, noise_size):
    random_values = np.random.randn(batch_size*noise_size)
    random_noise_batch = np.reshape(random_values, (batch_size, noise_size))
    return random_noise_batch'''

y=get_random_noise(100,100)
yx=generator_network.predict_on_batch(y)


# In[24]:


def upsampler(data):
    up_imgs=[]
    for i in range(len(data)):
        up_imgs.append(cv2.resize(data[i],(75,75),interpolation = cv2.INTER_AREA))
        
    return np.array(up_imgs)


# In[25]:


generated_images=upsampler(yx)


# In[26]:


real_images=upsampler(images2[:100])


# In[27]:


generated_images.shape, real_images.shape


# In[29]:



model=tensorflow.keras.applications.InceptionV3(include_top=False, pooling='avg', input_shape=(75,75,3))


# In[30]:


# example of calculating the frechet inception distance
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm

def calculate_fid(model,images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid


# In[32]:


score=calculate_fid(model,generated_images,real_images)


# In[33]:


score


# In[40]:


plt.imshow(yx[59])


# In[41]:


plt.figure(figsize=(10,10)) # specifying the overall grid size

for i in range(25):
    plt.subplot(5,5,i+1)    # the number of images in the grid is 5*5 (25)
    plt.imshow(yx[i])

plt.show()


# In[ ]:






## VAE

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[59]:


import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os


# In[60]:


os.environ["CUDA_VISIBLE_DEVICES"]="1"


# In[61]:


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


# In[62]:


latent_dim = 21

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()


# In[63]:


latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()


# In[64]:


class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


# In[65]:


(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255


# In[66]:


import cv2
import glob
images1 = [cv2.cvtColor(cv2.resize(cv2.imread(file),(28,28)),cv2.COLOR_BGR2GRAY) for file in glob.glob('IMagenet/tiny-imagenet-200/val/images/*.JPEG')]
images2t=[cv2.cvtColor(cv2.resize(cv2.imread(file),(28,28)),cv2.COLOR_BGR2GRAY) for file in glob.glob('IMagenet/tiny-imagenet-200/test/images/*.JPEG')]
images2=images2t[:9900]


# In[67]:


train_images=np.vstack((images1,images2))


# In[68]:


train_images.shape


# In[69]:


train_images=np.expand_dims(train_images, -1).astype("float32") / 255


# In[70]:


#train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

#train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]


# In[71]:


train_images.shape


# In[72]:


vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(train_images, epochs=300, batch_size=128)


# In[80]:


def sampler(N):
    generated_data=[]
    generated_labels=[]
    for i in range(N):
        generated_data.append(np.random.multivariate_normal(np.zeros(21),np.identity(21)))
    return (np.array(generated_data))    


# In[81]:


model=tf.keras.applications.InceptionV3(include_top=False, pooling='avg', input_shape=(75,75,3))


# In[82]:


# example of calculating the frechet inception distance
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm

def calculate_fid(model,images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid


# In[83]:


def upsampler(data):
    up_imgs=[]
    for i in range(len(data)):
        up_imgs.append(cv2.resize(data[i],(75,75),interpolation = cv2.INTER_AREA))
        
    return np.array(up_imgs)


# In[84]:


def gray_rgb(data):
    rgb_imgs=[]
    for i in range(len(data)):
        rgb_imgs.append(cv2.cvtColor(data[i],cv2.COLOR_GRAY2RGB))
    return np.array(rgb_imgs)


# In[85]:


samples=sampler(100)


# In[86]:


p=decoder.predict(samples)


# In[87]:


predicted_images=upsampler(p)


# In[88]:


predicted_images.shape


# In[89]:


predicted_images=gray_rgb(predicted_images)


# In[90]:


predicted_images.shape


# In[91]:


real_images=gray_rgb(images2[:100])
real_images=upsampler(real_images)


# In[92]:


score=calculate_fid(model,predicted_images,real_images)


# In[93]:


score


# In[94]:


import matplotlib.pyplot as plt
plt.imshow(p[76])


# In[95]:


plt.figure(figsize=(10,10)) # specifying the overall grid size

for i in range(25):
    plt.subplot(5,5,i+1)    # the number of images in the grid is 5*5 (25)
    plt.imshow(p[i])

plt.show()


## tSNE & PCA

In [ ]:
from numpy import load 
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from __future__ import print_function
from sklearn import decomposition
from sklearn.decomposition import PCA, IncrementalPCA
import time
import numpy as np
import pandas as pd
import numpy as np
import skimage.io as io
import numpy.linalg as linalg
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from numpy import linalg as LA
from skimage.util import img_as_float
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns


# In[2]:



data = load('bloodmnist.npz') 
lst = data.files 
Xtrain = data[lst[0]] 
Xtest = data[lst[1]] 
Xval = data[lst[2]] 
ytrain = data[lst[3]] 
ytest = data[lst[4]] 
yval = data[lst[5]]

Xtrain = data[lst[0]]
ytrain = data[lst[1]]
Xval = data[lst[2]]
yval = data[lst[3]]
Xtest = data[lst[4]]
ytest = data[lst[5]]

print(Xtrain.shape) 
print(Xtest.shape) 
print(Xval.shape) 
print(ytrain.shape) 
print(ytest.shape) 
print(yval.shape)


# In[3]:


Xtrain=Xtrain.reshape(Xtrain.shape[0],2352)
Xval=Xval.reshape(Xval.shape[0],2352)
Xtest = Xtest.reshape(Xtest.shape[0],2352)
print(Xtrain.shape) 
print(Xtest.shape) 
print(Xval.shape) 


# In[19]:


feat_cols = [ 'pixel'+str(i) for i in range(Xtrain.shape[1]) ]
df = pd.DataFrame(Xtrain,columns=feat_cols)
df['y'] = ytrain
df['label'] = df['y'].apply(lambda i: str(i))
X, y = None, None
print('Size of the dataframe: {}'.format(df.shape))


# In[7]:





# In[20]:


time_start = time.time()
pca_50 = PCA(n_components=50)
pca_result_50 = pca_50.fit_transform(df[feat_cols].values)
print('Cumulative explained variation for 50 principal components: {}'.format(np.sum(pca_50.explained_variance_ratio_)))
print('pca done! Time elapsed: {} seconds'.format(time.time()-time_start))


# In[21]:


df['pca-one'] = pca_result_50[:,0]
df['pca-two'] = pca_result_50[:,1] 
df['pca-three'] = pca_result_50[:,2]


# In[22]:


np.random.seed(42)
rndperm = np.random.permutation(df.shape[0])
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="y",
    palette=sns.color_palette("hls", 8),
    data=df.loc[rndperm,:],
    legend="full",
    alpha=0.3
)


# In[23]:


ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(
    xs=df.loc[rndperm,:]["pca-one"], 
    ys=df.loc[rndperm,:]["pca-two"], 
    zs=df.loc[rndperm,:]["pca-three"], 
    c=df.loc[rndperm,:]["y"], 
    cmap='tab10'
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.show()


# # tSNE

# In[24]:


time_start = time.time()
tsne = TSNE(n_components=3, verbose=0, perplexity=40, n_iter=300)
tsne_pca_results = tsne.fit_transform(pca_result_50)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))


# In[33]:


df['tsne-one'] = tsne_pca_results[:,0]
df['tsne-two'] = tsne_pca_results[:,1]
df['tsne-three'] = tsne_pca_results[:,2]


# In[35]:


np.random.seed(42)
rndperm = np.random.permutation(df.shape[0])
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-one", y="tsne-two",
    hue="y",
    palette=sns.color_palette("hls", 8),
    data=df.loc[rndperm,:],
    legend="full",
    alpha=0.3
)


# In[36]:


ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(
    xs=df.loc[rndperm,:]["tsne-one"], 
    ys=df.loc[rndperm,:]["tsne-two"], 
    zs=df.loc[rndperm,:]["tsne-three"], 
    c=df.loc[rndperm,:]["y"], 
    cmap='tab10'
)
ax.set_xlabel('tsne-one')
ax.set_ylabel('tsne-two')
ax.set_zlabel('tsne-three')
plt.show()


# In[37]:


data = load('bloodmnist.npz') 
lst = data.files 
Xtrain1 = data[lst[0]] 
Xtest1 = data[lst[1]] 
Xval1 = data[lst[2]] 
ytrain1 = data[lst[3]] 
ytest1 = data[lst[4]] 
yval1 = data[lst[5]]


print(Xtrain1.shape) 
print(Xtest1.shape) 
print(Xval1.shape) 
print(ytrain1.shape) 
print(ytest1.shape) 
print(yval1.shape)


# In[39]:


Xtrain1[1].shape


# In[44]:


io.imshow(Xtrain1[10])


# In[63]:


my_image=Xtrain1[10]
image_sum = my_image.sum(axis=2)
print(image_sum.shape)

new_image = image_sum/image_sum.max()
print(new_image.max())

#plt.figure(figsize=[12,8])
plt.imshow(new_image, cmap=plt.cm.gray)


# In[ ]:


newimage=


# In[66]:


ipca = IncrementalPCA(n_components=20)
image_recon = ipca.inverse_transform(ipca.fit_transform(new_image))

# Plotting the reconstructed image
#plt.figure(figsize=[12,8])
plt.imshow(image_recon,cmap = plt.cm.gray)
